In [1]:
from new_parser import construct, match

In [2]:
from utils import SocketReader

In [3]:
try:
    sock.close()
except:
    pass

sock = SocketReader("stun2.l.google.com", 19302, 'udp')

sock.connect(source_port=45677)

In [4]:
stun_header_spec = """
2==0
-14@message_type
-16@message_length
-32@magic_cookie
-96@transaction_id
-{message_length}@message(
    ...@attributes(
        16==0x20-16@xor_mapped_ip(
            8==0
            -8@family
            -16@port[int]
            -32@ip[ip]
        )
    )
)
"""

In [5]:
import random
magic_cookie = 0x2112A442 # constant
stun_header = {
    # +--+--+-+-+-+-+-+-+-+-+-+-+-+-+
    # |M |M |M|M|M|C|M|M|M|C|M|M|M|M|
    # |11|10|9|8|7|1|6|5|4|0|3|2|1|0|
    # +--+--+-+-+-+-+-+-+-+-+-+-+-+-+
    "message_type": 0b00000000000001,
    "message_length": 0,
    "magic_cookie": magic_cookie,
    "transaction_id": random.randrange(1<<96),
    'message': {'attributes': []},
}

In [6]:
stun_header_encoded = construct(stun_header_spec, stun_header)
stun_header_encoded

b'\x00\x01\x00\x00!\x12\xa4B\x13+\x0eU?\x14`U\x0f\x8c<\xa1'

In [7]:
sock.send(stun_header_encoded)

In [8]:
resp = sock.read_once()

In [9]:
res, rem = match(stun_header_spec, resp)
res

{'message_type': 0b00000100000001,
 'magic_cookie': 0x2112a442,
 'transaction_id': 0x132b0e553f1460550f8c3ca1,
 'message': {'attributes': [{'xor_mapped_ip': {'family': 1,
     'port': 37759,
     'ip': '174.160.236.184'}}]}}

In [10]:
port = res['message']['attributes'][0]['xor_mapped_ip']['port'].value('int') ^ (magic_cookie >> 16)
port

45677

In [11]:
ip_number = res['message']['attributes'][0]['xor_mapped_ip']['ip'].value('int') ^ (magic_cookie)
ip = '%d.%d.%d.%d'%tuple([(ip_number >> (8 * s)) & (255) for s in range(3, -1, -1)])
ip

'143.178.72.250'

In [12]:
sock.close()